In [40]:
import snowflake.connector
import pandas as pd
import numpy as np

In [20]:
ctx = snowflake.connector.connect(
    user='leon.zhao@offerupnow.com',
    password='Steel24ly@offerup',
    account='offerup'
    )
cs = ctx.cursor()
try:
    cs.execute("SELECT current_version()")
    one_row = cs.fetchone()
    print(one_row[0])
finally:
    cs.close()


3.12.0


In [21]:
results = ctx.cursor().execute("USE WAREHOUSE ANALYTICS_WH").fetchall()

In [16]:
def get_Data():
    from snowflake.connector import DictCursor
    # Querying data by DictCursor
    cur = ctx.cursor(DictCursor)
    try:
        df1 = []
        data = cur.execute("select * from ou_users.leon.active_seller_201812")
        data = pd.DataFrame(data.fetchall())
        df1 = data.iloc[-30:]
        print(df1.tail(6))
    finally:
        cur.close()
    return data.to_csv('new_data.csv')

In [17]:
get_Data()

         ACTIVE_ITEMS_FROM_PURCHASERS  \
6662416                             0   
6662417                             0   
6662418                             0   
6662419                             0   
6662420                             0   
6662421                             0   

         ACTIVE_ITEMS_WITH_PURCHASE_FROM_ADVERTISER EMAIL       MONTH  \
6662416                                           0  None  2018-12-01   
6662417                                           0  None  2018-12-01   
6662418                                           0  None  2018-12-01   
6662419                                           0  None  2018-12-01   
6662420                                           0  None  2018-12-01   
6662421                                           0  None  2018-12-01   

        PLATFORM   USER_ID  VIEWED_PROMO_PAGE_NO_PURCHASE  
6662416      iOS  46958791                              1  
6662417      iOS  44736431                              1  
6662418      iOS  12

In [22]:
import pandas as pd
from snowflake.connector import DictCursor
cur = ctx.cursor(DictCursor)

df1 = []
revenue_data = cur.execute("WITH final AS \
                           (WIth store_sales_data as \
                           (SELECT CONVERT_TIMEZONE('America/Los_Angeles', to_timestamp_tz(ORDER_CHARGED_TIMESTAMP))::DATE AS date, \
                                product_title, \
                                sku_id AS sku, \
                                CASE WHEN lower(sku_id) LIKE '%bump%' THEN 'Bumps' \
                                  WHEN lower(sku_id) LIKE '%sellerad%' OR lower(sku_id) LIKE '%feature%' THEN 'Seller Ads' \
                                  WHEN lower(sku_id) LIKE '%subscription.promote%' THEN 'Seller Ads Subscriptions' \
                                  WHEN lower(sku_id) LIKE '%listing_fee%' THEN 'Listing Fees' \
                                  ELSE 'Other' \
                                END AS purchase_type_group, \
                                item_price, \
                                'android' AS platform, \
                                COUNT(CASE WHEN financial_status = 'Charged' THEN 1 WHEN financial_status = 'Refund' THEN -1 ELSE 0 END) AS units, \
                                COUNT(CASE WHEN financial_status = 'Charged' THEN 1 ELSE NULL END) AS charged_units, \
                                COUNT(CASE WHEN financial_status = 'Refund' THEN 1 ELSE NULL END) AS refunded_units, \
                                SUM(CASE WHEN financial_status = 'Charged' THEN item_price ELSE 0.00 END) AS charged_revenue, \
                                SUM(CASE WHEN financial_status = 'Refund' THEN item_price ELSE 0.00 END) AS refunded_revenue, \
                                SUM(CASE WHEN financial_status = 'Charged' THEN item_price WHEN financial_status = 'Refund' THEN -1 * item_price ELSE 0.00 END) AS revenue \
                          FROM app_stores.public.google_sales \
                          GROUP BY 1, 2, 3, 4, 5, 6 \
                         \
                          UNION ALL \
                         \
                          SELECT begin_date AS date, \
                                title AS product_title, \
                                sku, \
                                CASE WHEN lower(sku) LIKE '%bump%' THEN 'Bumps' \
                                  WHEN lower(sku) LIKE '%feature%' OR  lower(sku) LIKE '%sellerad%' THEN 'Seller Ads' \
                                  WHEN lower(sku) LIKE '%subscription.promote%' THEN 'Seller Ads Subscriptions' \
                                  WHEN lower(sku) LIKE '%listing_fee%' THEN 'Listing Fees' \
                                  ELSE 'Other' \
                                END AS purchase_type_group, \
                                customer_price AS item_price, \
                                'iOS' AS platform, \
                                SUM(CASE WHEN customer_price > 0 THEN units WHEN customer_price < 0 THEN units ELSE 0 END) AS units, \
                                SUM(CASE WHEN customer_price > 0 THEN units ELSE 0 END) AS charged_units, \
                                SUM(CASE WHEN customer_price < 0  THEN units ELSE 0 END) AS refunded_units, \
                                SUM(CASE WHEN customer_price > 0  THEN customer_price * units ELSE 0.00 END) AS charged_revenue, \
                                SUM(CASE WHEN customer_price < 0  THEN -1 * customer_price * units ELSE 0.00 END) AS refunded_revenue, \
                                SUM(CASE WHEN customer_price > 0  THEN customer_price * units WHEN customer_price < 0  THEN -1 * customer_price * units ELSE 0.00 END) AS revenue \
                          FROM app_stores.public.apple_sales_summary \
                          GROUP BY 1, 2, 3, 4, 5, 6), \
                          user as \
                          (WITH first_messages AS \
                           (select buyer_id, \
                                   min(date_created) as first_message_sent \
                           from oubi.rds_public.item_itemmessagethread \
                           group by 1), \
                           first_posts AS \
                           (select owner_id, \
                                   min(created) as first_item_posted \
                           from oubi.rds_public.item_item \
                           group by 1) \
                          SELECT u.accepted_tos_id \
                                ,u.avatar \
                                ,u.active_changed \
                                ,u.city \
                                ,u.created \
                                ,fp.first_item_posted \
                                ,fm.first_message_sent \
                                ,u.gender \
                                ,u.id \
                                ,u.modified \
                                ,u.newsletter_changed \
                                ,u.payment_enabled \
                                ,u.payment_enabled_changed \
                                ,u.seller_account_active \
                                ,u.seller_account_id \
                                ,u.seller_approved_or_denied \
                                ,u.seller_signed_up \
                                ,u.state \
                                ,u.user_id \
                                ,u.verified \
                                ,u.verified_by_id \
                                ,u.verified_date \
                                ,u.welcomed_changed \
                                ,u.zipcode \
                                ,case when u.facebook_id is not null then TRUE else FALSE end as facebook_connected \
                                ,au.is_active \
                                ,au.username \
                              from oubi.rds_public.account_userprofile u \
                              inner join oubi.rds_public.auth_user au on au.id=u.user_id \
                              left join first_messages fm on fm.buyer_id=u.user_id \
                              left join first_posts fp on fp.owner_id=u.user_id) \
                           SELECT CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', disbursed_at::timestamp_ntz)::DATE AS date, \
                               CASE WHEN header_origin = 'android' THEN 'Android' \
                               WHEN header_origin = 'ios' THEN 'iOS' \
                               WHEN header_origin = 'iphone' THEN 'iOS' \
                               WHEN header_origin = 'ipad' THEN 'iOS' \
                                   ELSE header_origin \
                               END AS platform, \
                               CASE WHEN city = 'Seattle-Tacoma, WA' OR city = 'Sacramento-Stockton-Modesto, CA' THEN city ELSE 'Other' END AS dimension_1, \
                               'dma' AS dimension_name, \
                               'payments_old' AS revenue_source, \
                               SUM(ou_revenue) AS gross_revenue, \
                               SUM(payment_amount) AS payment_amount, \
                               COUNT(DISTINCT item_id::VARCHAR(128) || ' - ' || payer_id::VARCHAR(128)) AS transactions \
                           FROM \
                           ( select p.id as id \
                                   ,p.amount as payment_amount \
                                   ,p.commision_amount as ou_revenue \
                                   ,p.settled_at \
                                   ,p.submitted_for_settlement_at \
                                   ,p.offerup_debit_amount \
                                   ,p.buyer_debit_amount \
                                   ,p.fee_collected_from_buyer_amount \
                                   ,p.fee_collected_from_offerup_amount \
                                   ,p.commision_amount \
                                   ,p.commision_rate \
                                   ,oi.item_id \
                                   ,p.created_at \
                                   ,p.authorized_at \
                                   ,p.canceled_at \
                                   ,p.refunded_at \
                                   ,p.disbursed_at \
                                   ,p.canceler_id \
                                   ,p.payment_status \
                                   ,p.payer_id \
                                   ,p.payee_id \
                                   ,p.cancel_reason \
                                   ,pm.card_type \
                                   ,pm.display_name \
                                   ,dma.dma_name as city \
                                   ,pay.header_origin \
                           from oubi.public.payments_payment p \
                           left join events_prod.public.PAYMENT_INITIALIZED_EVENT AS pay \
                               ON p.payer_id = pay.payment_payer_id \
                                   AND p.id = pay.payment_id \
                           inner join oubi.public.payments_order o on o.id=p.order_id \
                           inner join oubi.public.payments_orderitem oi on oi.order_id=o.id \
                           left join oubi.pg.item_dma dma on dma.item_id=oi.item_id \
                           left join oubi.public.payments_paymentmethod pm on pm.id=p.payment_method_used_id \
                           where p.payment_gateway_id=2) AS a \
                           WHERE disbursed_at IS NOT NULL \
                           GROUP BY 1, 2, 3, 4, 5 \
                           \
                           UNION ALL \
                            \
                            SELECT date, \
                            CASE WHEN platform = 'android' THEN 'Android' ELSE platform END AS platform, \
                            ROUND(item_price,2)::VARCHAR(5) AS dimension_1, \
                            'price' AS dimension_name, \
                            'bump' AS revenue_source, \
                            SUM(revenue) AS gross_revenue, \
                            0 AS payment_amount, \
                            SUM(units) AS transactions \
                            FROM store_sales_data \
                            WHERE lower(sku) LIKE '%bump%' \
                            GROUP BY 1, 2, 3 ,4, 5 \
                            \
                            UNION ALL \
                            \
                            SELECT date, \
                            CASE WHEN platform = 'android' THEN 'Android' ELSE platform END AS platform, \
                            name AS dimension_1, \
                            'ad placement' AS dimension_name, \
                            'external ads' AS revenue_source, \
                            SUM(CASE WHEN ad_network = 'Bing' THEN estimated_gross_revenue *.69 WHEN ad_network = 'Google' THEN estimated_gross_revenue * .55 ELSE estimated_gross_revenue END) AS gross_revenue, \
                            0 AS payment_amount, \
                            SUM(clicks) AS transactions \
                            FROM \
                            ( \
                            SELECT * \
                            FROM marketing.public.in_app_ads_data \
                            ) AS a \
                            GROUP BY 1, 2, 3, 4, 5 \
                            \
                            UNION ALL \
                            \
                            SELECT activity_date AS date, \
                            CASE WHEN a.ad_unit_name LIKE '%_andr_%' THEN 'Android' \
                            WHEN a.ad_unit_name LIKE '%_ios_%' THEN 'iOS' \
                            WHEN a.ad_unit_name LIKE '%web%' THEN 'Web' \
                            ELSE 'Other' \
                            END AS platform, \
                            COALESCE(b.ad_unit_name_lookup, 'Unknown') AS dimension_1, \
                            'ad placement' AS dimension_name, \
                            'external ads' AS revenue_source, \
                            SUM(CASE WHEN activity_date < '2018-03-06' THEN estimated_revenue * 0.69 ELSE estimated_revenue * 0.84 END) AS gross_revenue, \
                            0 AS payment_amount, \
                            SUM(clicks) AS transactions \
                            FROM bing.public.adunit_metrics AS a \
                            LEFT OUTER JOIN marketing.public.bing_adunit_name_lookup AS b \
                            ON a.ad_unit_name = b.ad_unit_name \
                            WHERE activity_date >= '2017-08-01' \
                            GROUP BY 1, 2, 3, 4, 5 \
                            \
                            UNION ALL \
                            \
                            SELECT date, \
                            CASE WHEN platform = 'android' THEN 'Android' ELSE platform END AS platform, \
                            CASE WHEN sku LIKE 'com.offerup.consumer.feature.%' THEN substring(substring(sku, 30, length(sku) - 29), 0, position('.' IN substring(sku, 30, length(sku) - 29))-1) \
                            WHEN sku LIKE 'com.offerup.iab.feature.%' THEN substring(substring(sku, 25), 0,position('.' IN substring(sku, 25)) - 1) END AS dimension_1, \
                            'promotion length' AS dimension_name, \
                            'seller ads' AS revenue_source, \
                            SUM(revenue) AS gross_revenue, \
                            0 AS payment_amount, \
                            SUM(units) AS transactions \
                            FROM store_sales_data \
                            WHERE lower(sku) LIKE '%sellerad%' OR lower(sku) LIKE '%feature%' \
                            GROUP BY 1, 2, 3, 4, 5 \
                            \
                            UNION ALL \
                            \
                            SELECT date, \
                            CASE WHEN platform = 'android' THEN 'Android' ELSE platform END AS platform, \
                            CASE WHEN sku LIKE 'com.offerup.consumer.subscription.promote.%' THEN CASE WHEN position('.' IN substring(sku, 43)) = '0' THEN substring(sku, 43, length(sku) - 42) ELSE substring(substring(sku, 43), 0,position('.' IN substring(sku, 43)) - 1) END END AS dimension_1, \
                            'promotion length' AS dimension_name, \
                            'seller ads subscriptions' AS revenue_source, \
                            SUM(revenue) AS gross_revenue, \
                            0 AS payment_amount, \
                            SUM(units) AS transactions \
                            FROM store_sales_data \
                            WHERE lower(sku) LIKE '%subscription.promote%' \
                            GROUP BY 1, 2, 3, 4, 5 \
                            \
                            UNION ALL \
                            \
                            SELECT date, \
                            CASE WHEN platform = 'android' THEN 'Android' ELSE platform END AS platform, \
                            CASE WHEN sku LIKE 'com.offerup.consumer.listing_fee.%' THEN CASE WHEN position('.' IN substring(sku, 34)) = '0' THEN substring(sku, 34, length(sku) - 33) ELSE substring(substring(sku, 34), 0,position('.' IN substring(sku, 34)) - 1) END END AS dimension_1, \
                            'listing fee type' AS dimension_name, \
                            'listing fees' AS revenue_source, \
                            SUM(revenue) AS gross_revenue, \
                            0 AS payment_amount, \
                            SUM(units) AS transactions \
                            FROM store_sales_data \
                            WHERE lower(sku) LIKE '%com.offerup.consumer.listing_fee%' \
                            GROUP BY 1, 2, 3, 4, 5 \
                            \
                            UNION ALL \
                            \
                            SELECT ship_date as date, \
                            NULL as platform, \
                            NULL as dimension_1, \
                            NULL as dimension_name, \
                            'payments' as revenue_source, \
                            sum(gross_payment_revenue) as gross_revenue, \
                            0 as payment_amount, \
                            count(distinct item_id) as transactions \
                            FROM marketing.public.shipping_revenue \
                            GROUP BY 1 \
                            \
                            UNION ALL \
                            \
                            SELECT ship_date as date, \
                            NULL as platform, \
                            NULL as dimension_1, \
                            NULL as dimension_name, \
                            'shipping' as revenue_source, \
                            sum(shipping_rev) as gross_revenue, \
                            0 as payment_amount, \
                            count(distinct item_id) as transactions \
                            FROM marketing.public.shipping_revenue \
                            GROUP BY 1 \
                            \
                            UNION ALL \
                            \
                            SELECT CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', created_at::timestamp_ntz)::DATE AS date, \
                            CASE WHEN header_origin = 'android' THEN 'Android' \
                            WHEN header_origin = 'ios' THEN 'iOS' \
                            WHEN header_origin = 'iphone' THEN 'iOS' \
                            WHEN header_origin = 'ipad' THEN 'iOS' \
                            ELSE header_origin \
                            END AS platform, \
                            NULL AS dimension_1, \
                            NULL AS dimension_name, \
                            'cashless_payments' AS revenue_source, \
                            SUM(ou_revenue) AS gross_revenue, \
                            SUM(payment_amount) AS payment_amount, \
                            COUNT(DISTINCT item_id::VARCHAR(128) || ' - ' || payer_id::VARCHAR(128)) AS transactions \
                            FROM \
                            ( \
                            select p.id as id \
                            ,p.payer_id \
                            ,p.created_at \
                            ,p.payment_status \
                            ,p.amount as payment_amount \
                            ,p.commision_amount + p.additional_fees_amount as ou_revenue \
                            ,pay.header_origin \
                            ,oi.item_id \
                            from oubi.public.payments_payment p \
                            left join events_prod.public.PAYMENT_INITIALIZED_EVENT AS pay \
                            ON p.payer_id = pay.payment_payer_id \
                            AND p.id = pay.payment_id \
                            join oubi.public.payments_orderitem oi \
                            on oi.order_id = p.order_id \
                            where payment_source = 'P2P' \
                            ) AS a \
                            WHERE payment_status in (5,6,20) \
                            GROUP BY 1, 2, 3, 4, 5 \
                            \
                            UNION ALL \
                            \
                            SELECT CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', settled_at::timestamp_ntz)::DATE AS date, \
                            'Web' AS platform, \
                            payment_amount::VARCHAR(5) AS dimension_1, \
                            'item price' AS dimension_name, \
                            'web internal ads purchases' AS revenue_source, \
                            SUM(payment_amount) AS gross_revenue, \
                            SUM(payment_amount) AS payment_amount, \
                            COUNT(1) AS transactions \
                            FROM \
                            ( \
                            select p.id as id \
                            ,p.amount as payment_amount \
                            ,p.commision_amount as ou_revenue \
                            ,p.settled_at \
                            ,p.submitted_for_settlement_at \
                            ,p.offerup_debit_amount \
                            ,p.buyer_debit_amount \
                            ,p.fee_collected_from_buyer_amount \
                            ,p.fee_collected_from_offerup_amount \
                            ,p.commision_amount \
                            ,p.commision_rate \
                            ,oi.item_id \
                            ,p.created_at \
                            ,p.authorized_at \
                            ,p.canceled_at \
                            ,p.refunded_at \
                            ,p.disbursed_at \
                            ,p.canceler_id \
                            ,p.payment_status \
                            ,p.payer_id \
                            ,p.payee_id \
                            ,p.cancel_reason \
                            ,pm.card_type \
                            ,pm.display_name \
                            ,dma.dma_name as city \
                            ,pay.header_origin \
                            from oubi.public.payments_payment p \
                            left join events_prod.public.PAYMENT_INITIALIZED_EVENT AS pay \
                            ON p.payer_id = pay.payment_payer_id \
                            AND p.id = pay.payment_id \
                            inner join oubi.public.payments_order o on o.id=p.order_id \
                            inner join oubi.public.payments_orderitem oi on oi.order_id=o.id \
                            left join oubi.pg.item_dma dma on dma.item_id=oi.item_id \
                            left join oubi.public.payments_paymentmethod pm on pm.id=p.payment_method_used_id \
                            where payment_source = 'SellerAds' \
                            ) AS a \
                            WHERE settled_at IS NOT NULL \
                            AND canceled_at IS NULL \
                            AND refunded_at IS NULL \
                            GROUP BY 1, 2, 3, 4, 5 \
                            \
                            UNION ALL \
                            \
                            SELECT date, \
                            CASE WHEN header_origin = 'android' THEN 'Android' \
                            WHEN header_origin = 'ios' THEN 'iOS' \
                            WHEN header_origin = 'iphone' THEN 'iOS' \
                            WHEN header_origin = 'ipad' THEN 'iOS' \
                            ELSE header_origin \
                            END AS platform, \
                            payment_price::VARCHAR(4) AS dimension_1, \
                            'price' AS dimension_name, \
                            'bump' AS revenue_source, \
                            SUM(payment_price) AS gross_revenue, \
                            0 AS payment_amount, \
                            COUNT(DISTINCT header_unique_id) AS transactions \
                            FROM \
                            (SELECT CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', header_timestamp::TIMESTAMP_ntz)::DATE AS date, \
                            ROUND((case when bump_sku='com.offerup.iab.bump' then 0.99 \
                            when bump_sku='com.offerup.iab.bump.optiontwo' then 1.99 \
                            when bump_sku='com.offerup.iab.bump.optionthree' then 2.99 \
                            when bump_sku='com.offerup.iab.bump.optionfour' then 3.99 \
                            when bump_sku='com.offerup.iab.bump.optionfive' then 4.99 \
                            when bump_sku='com.offerup.iab.bump.optionsix' then 5.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump04' then 0.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump05' then 1.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump06' then 2.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump07' then 3.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump08' then 4.99 \
                            when bump_sku='com.offerup.iphone.consumer.bump09' then 5.99 \
                            when header_unique_id='e95e244296bc4685966972d425dcf021' then 3.99 \
                            when header_device_id='c5cf27a2b15114a4' then 2.99 \
                            when header_device_id='e8fb7c40c4d435cc' then 1.99 \
                            else payment_price \
                            end)::NUMERIC(5,4),2) as payment_price, \
                            item_id, \
                            actor_id, \
                            header_unique_id, \
                            header_origin \
                            FROM events_prod.public.item_bumped_event_with_pmt_fix AS item_bumped_event \
                            LEFT OUTER JOIN user AS seller_info \
                            ON seller_info.user_id=item_bumped_event.actor_id \
                            WHERE (item_bumped_event.actor_type <> 'ou_staff' OR item_bumped_event.actor_type IS NULL) \
                            AND CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', header_timestamp::TIMESTAMP_ntz)::DATE < '2017-10-31' \
                            AND header_origin IN ('ios', 'iphone', 'ipad') \
                            \
                            UNION ALL \
                            \
                            SELECT CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', header_timestamp::TIMESTAMP_ntz)::DATE AS date, \
                            ROUND((case when promo_sku='com.offerup.consumer.bump.tier1.mid.1_99' then 1.99 \
                            when promo_sku='com.offerup.consumer.bump.tier2.low.3_99' then 3.99 \
                            else (promo_price_in_microunits / 1000000)::NUMERIC(7,4) \
                            end)::NUMERIC(7,4),2) as payment_price, \
                            item_id, \
                            header_user_id AS actor_id, \
                            header_unique_id, \
                            CASE WHEN header_timestamp >= '2018-09-08' THEN header_origin \
                            WHEN promo_sku LIKE '%com.offerup.consumer%' THEN 'iOS' \
                            WHEN promo_sku LIKE '%com.offerup.iab.bump%' THEN 'android' \
                            ELSE header_origin \
                            END AS platform \
                            FROM events_prod.public.item_promoted_event AS a \
                            WHERE promo_type = 'BUMP' \
                            AND  CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', header_timestamp::TIMESTAMP_ntz)::DATE < '2017-10-31' \
                            AND promo_sku LIKE '%com.offerup.consumer%' \
                            ) AS a \
                            WHERE payment_price > 0 \
                            GROUP BY 1, 2, 3 ,4, 5 \
                            \
                            UNION ALL \
                            \
                            SELECT date, \
                            CASE WHEN platform = 'android' THEN 'Android' \
                            WHEN platform = 'ios' THEN 'iOS' \
                            WHEN platform = 'iphone' THEN 'iOS' \
                            WHEN platform = 'ipad' THEN 'iOS' \
                            ELSE platform \
                            END AS platform, \
                            LEFT(promo_length, position('D', promo_length) - 1) || 'day' AS dimension_1, \
                            'promotion length' AS dimension_name, \
                            'seller ads' AS revenue_source, \
                            SUM(payment_price) AS gross_revenue, \
                            0 AS payment_amount, \
                            COUNT(DISTINCT header_unique_id) AS transactions \
                            FROM \
                            (SELECT CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', header_timestamp::TIMESTAMP_ntz)::DATE AS date, \
                            ROUND((case when promo_sku='com.offerup.consumer.sellerad799' then 7.99 \
                            when promo_sku='com.offerup.consumer.sellerad1299' then 12.99 \
                            when promo_sku='com.offerup.consumer.sellerad1999' then 19.99 \
                            when promo_sku='com.offerup.iab.feature.3day.medium.7_99' then 7.99 \
                            when promo_sku='com.offerup.iab.feature.7day.medium.12_99' then 12.99 \
                            when promo_sku='com.offerup.iab.feature.14day.medium.19_99' then 19.99 \
                            else (promo_price_in_microunits / 1000000)::NUMERIC(7,4) \
                            end)::NUMERIC(7,4),2) as payment_price, \
                            item_id, \
                            header_user_id AS actor_id, \
                            header_unique_id, \
                            CASE WHEN header_timestamp >= '2018-09-08' THEN header_origin \
                            WHEN promo_sku LIKE '%com.offerup.consumer%' THEN 'iOS' \
                            WHEN promo_sku LIKE '%com.offerup.iab%' THEN 'android' \
                            ELSE header_origin \
                            END AS platform, \
                            SUBSTRING(promo_type, length('SELLER_AD_') + 1, length(promo_type) - length('SELLER_AD_')) AS promo_length \
                            FROM events_prod.public.item_promoted_event AS a \
                            WHERE promo_type <> 'BUMP' \
                            AND CONVERT_TIMEZONE('UTC', 'America/Los_Angeles', header_timestamp::TIMESTAMP_ntz)::DATE < '2017-10-31' \
                            AND promo_sku LIKE '%com.offerup.consumer%' \
                            ) AS a \
                            GROUP BY 1, 2, 3, 4, 5) \
                            SELECT date_trunc('month', date) as date_month, \
                            platform, \
                            revenue_source, \
                            round(sum(gross_revenue), 0) as gross_revenue \
                            FROM final \
                            WHERE revenue_source in ('seller ads subscriptions', 'bump', 'seller ads', 'external ads', 'web internal ads purchases') \
                            AND platform in ('iOS', 'Android', 'Web') \
                            group by 1, 2, 3 \
                            order by 1 asc, 2 asc ")
revenue_data = pd.DataFrame(revenue_data.fetchall())
df1 = revenue_data.iloc[-30:]
print(df1.tail())


    DATE_MONTH  GROSS_REVENUE PLATFORM              REVENUE_SOURCE
181 2019-01-01          13868      Web  web internal ads purchases
182 2019-01-01         243195      iOS                  seller ads
183 2019-01-01         955541      iOS                external ads
184 2019-01-01         124831      iOS                        bump
185 2019-01-01         398081      iOS    seller ads subscriptions


In [25]:
revenue_data.T

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
DATE_MONTH,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00,2016-08-01 00:00:00,2016-09-01 00:00:00,2016-09-01 00:00:00,2016-10-01 00:00:00,...,2019-01-01 00:00:00,2019-01-01 00:00:00,2019-01-01 00:00:00,2019-01-01 00:00:00,2019-01-01 00:00:00,2019-01-01 00:00:00,2019-01-01 00:00:00,2019-01-01 00:00:00,2019-01-01 00:00:00,2019-01-01 00:00:00
GROSS_REVENUE,1,658,207,13466,15228,22569,36974,64112,108421,4388,...,160191,1150613,58881,116430,47647,13868,243195,955541,124831,398081
PLATFORM,Android,Android,iOS,Android,iOS,Android,iOS,Android,iOS,Android,...,Android,Android,Android,Android,Web,Web,iOS,iOS,iOS,iOS
REVENUE_SOURCE,bump,bump,bump,bump,bump,bump,bump,bump,bump,external ads,...,seller ads subscriptions,external ads,bump,seller ads,external ads,web internal ads purchases,seller ads,external ads,bump,seller ads subscriptions


In [34]:
revenue_data.pivot_table(index=['PLATFORM', 'REVENUE_SOURCE'], values=['GROSS_REVENUE'], columns=['DATE_MONTH'], aggfunc=[np.sum])

sum                        \
                                    GROSS_REVENUE                         
DATE_MONTH                             2016-05-01 2016-06-01 2016-07-01   
PLATFORM REVENUE_SOURCE                                                   
Android  bump                                 1.0      658.0    13466.0   
         external ads                         NaN        NaN        NaN   
         seller ads                           NaN        NaN        NaN   
         seller ads subscriptions             NaN        NaN        NaN   
Web      external ads                         NaN        NaN        NaN   
         web internal ads purchases           NaN        NaN        NaN   
iOS      bump                                 NaN      207.0    15228.0   
         external ads                         NaN        NaN        NaN   
         seller ads                           NaN        NaN        NaN   
         seller ads subscriptions             NaN        NaN        NaN   

                                                                      \
                                                                       
DATE_MONTH                          2016-08-01 2016-09-01 2016-10-01   
PLATFORM REVENUE_SOURCE                                                
Android  bump                          22569.0    64112.0    67471.0   
         external ads                      NaN        NaN     4388.0   
         seller ads                        NaN        NaN        NaN   
         seller ads subscriptions          NaN        NaN        NaN   
Web      external ads                      NaN        NaN        NaN   
         web internal ads purchases        NaN        NaN        NaN   
iOS      bump                          36974.0   108421.0   119856.0   
         external ads                      NaN        NaN        NaN   
         seller ads                        NaN        NaN        NaN   
         seller ads subscriptions          NaN        NaN        NaN   

                                                                      \
                                                                       
DATE_MONTH                          2016-11-01 2016-12-01 2017-01-01   
PLATFORM REVENUE_SOURCE                                                
Android  bump                          66629.0    70418.0    67052.0   
         external ads                   6341.0     6239.0     6638.0   
         seller ads                        NaN        NaN        NaN   
         seller ads subscriptions          NaN        NaN        NaN   
Web      external ads                      NaN        NaN        NaN   
         web internal ads purchases        NaN        NaN        NaN   
iOS      bump                         123578.0   131755.0   137809.0   
         external ads                      NaN      371.0     4441.0   
         seller ads                        NaN        NaN        NaN   
         seller ads subscriptions          NaN        NaN        NaN   

                                                  ...                 \
                                                  ...                  
DATE_MONTH                          2017-02-01    ...     2018-04-01   
PLATFORM REVENUE_SOURCE                           ...                  
Android  bump                          69642.0    ...       135745.0   
         external ads                   8298.0    ...      1473426.0   
         seller ads                        NaN    ...       115289.0   
         seller ads subscriptions          NaN    ...            NaN   
Web      external ads                      NaN    ...        93285.0   
         web internal ads purchases        NaN    ...            NaN   
iOS      bump                         138267.0    ...       311689.0   
         external ads                   9167.0    ...      1284314.0   
         seller ads                        NaN    ...       237467.0   
         seller ads subscriptions        

In [1]:
import findspark
findspark.init()
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples
print(pi)
sc.stop()

3.14207312
